In [94]:
import pandas as pd
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [95]:
files = os.listdir('data/brenda')
files

['result_inhibitors.csv',
 'result_inhibitors_filtrado.csv',
 'result_kcat_km.csv',
 'result_kcat_km_filtrado .csv',
 'result_ki+filtrado .csv',
 'result_ki.csv',
 'result_turnover.csv',
 'result_turnover_filtrado.csv']

## NaN e Null

In [123]:
data_files = {k[:-4]: pd.read_csv(f'data/brenda/{v}', sep=';') for k,v in zip(files,files)}
data_files

{'result_inhibitors.csv':      Unnamed: 0 literature inhibitor   \
 0           300              [657241]   
 1           519              [657241]   
 2           382              [657241]   
 3           200              [684493]   
 4           577              [684493]   
 5           616              [684493]   
 6           359              [664636]   
 7           573              [664636]   
 8           199              [664636]   
 9           269              [664636]   
 10          339              [664636]   
 11          493              [664636]   
 12          510              [734294]   
 13          508              [734294]   
 14          530              [734294]   
 15           61              [719045]   
 16          498              [734294]   
 17          150              [734294]   
 18          229              [663641]   
 19          291              [711202]   
 20          569              [663641]   
 21          605              [663641]   
 22      

In [122]:
cols = []
for data in data_files:
    print('***isnull:***\n', data.isnull().sum(), end='\n\n')
    print('***NaN***\n', data.isna().sum(), end='\n\n')
    print(data.shape, end='\n\n')
    cols.append(data.columns)
    print(40*'-', end='\n\n')

***isnull:***
 Unnamed: 0                       0
literature inhibitor             0
inhibitor                        0
commentary inhibitor           405
organism                         0
ecNumber                         0
ligandStructureId inhibitor      0
CLASS                           27
Unnamed: 8                     691
dtype: int64

***NaN***
 Unnamed: 0                       0
literature inhibitor             0
inhibitor                        0
commentary inhibitor           405
organism                         0
ecNumber                         0
ligandStructureId inhibitor      0
CLASS                           27
Unnamed: 8                     691
dtype: int64

(692, 9)

----------------------------------------

***isnull:***
 Unnamed: 0                       0
literature inhibitor             0
inhibitor                        0
commentary inhibitor           279
organism                         0
ecNumber                         0
ligandStructureId inhibitor      0
CLAS

## Feature por file

In [118]:
listas_normalizadas = [sorted([item.lower() for item in sublista]) for sublista in cols]

iguais = all(sublista == listas_normalizadas[0] for sublista in listas_normalizadas)

col_file = pd.concat([pd.Series(listas_normalizadas[0]),pd.Series(listas_normalizadas[0]),
           pd.Series(listas_normalizadas[2]),pd.Series(listas_normalizadas[3]),
           pd.Series(listas_normalizadas[4]),pd.Series(listas_normalizadas[5]),
           pd.Series(listas_normalizadas[6]),pd.Series(listas_normalizadas[7])], axis=1)
col_file.columns = [f[:-4] for f in files]
col_file

,result_inhibitors,result_inhibitors_filtrado,result_kcat_km,result_kcat_km_filtrado,result_ki+filtrado,result_ki,result_turnover,result_turnover_filtrado
0,class,class,class,class,class,class,class,class
1,commentary inhibitor,commentary inhibitor,commentary kcat/km,commentary kcat/km,commentary ki,commentary ki,commentary turnover,commentary turnover
2,ecnumber,ecnumber,kcatkmvalue,kcatkmvalue,inhibitor ki,inhibitor ki,ligandstructureid turnover,ligandstructureid turnover
3,inhibitor,inhibitor,kcatkmvaluemaximum,kcatkmvaluemaximum,kivalue,kivalue,literature turnover,literature turnover
4,ligandstructureid inhibitor,ligandstructureid inhibitor,ligandstructureid kcat/km,ligandstructureid kcat/km,kivaluemaximum,kivaluemaximum,organism,organism
5,literature inhibitor,literature inhibitor,literature kcat/km,literature kcat/km,ligandstructureid ki,ligandstructureid ki,substrate turnover,substrate turnover
6,organism,organism,organism,organism,literature ki,literature ki,turnovernumber,turnovernumber
7,unnamed: 0,unnamed: 0,substrate,substrate,organism,organism,turnovernumbermaximum,turnovernumbermaximum
8,unnamed: 8,unnamed: 8,NaN,NaN,NaN,NaN,NaN,NaN
